# AEMET diario-extraer series
***

_Autor:_    __Jesús Casado__ <br> _Revisión:_ __16/04/2020__ <br>

__Introducción__<br>
Funciones para generar series a partir de los datos brutos diarios de AEMET.

__Cosas a corregir__ <br>

In [8]:
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook')
from calendar import monthrange

### Precipitación

In [9]:
def AEMET_Pdiaria(estacion, provincia):
    """Extrae la serie diaria de precipitación de la AEMET.
    
    Parámetros:
    -----------
    estacion:  string. Código de la estación
    provinvia: string. Provincia a la que pertenece la estación. Define el archivo csv donde acceder a los datos, por lo que debe replicar 
                       el nombre del archivo, es decir, primera letra en mayúsucula y sin ninguna tilde.
                       
    Salidas:
    --------
    serie:     pd.Series. Serie diaria de precipitación en mm
    atributos: pd.Series. Atributos de la estación: coordenadas, altitud...
    """

    # cargar datos burtos de la provincia
    ruta = 'G:/CLIMA/02_HYDRO-CLIMATE/AEMET/precipitacion/diario/'
    raw = pd.read_csv(ruta + provincia + '.csv', sep=';', encoding='latin1')
    # extraer datos de la estación
    data = raw.loc[raw.INDICATIVO == estacion,:]
    
    # ATRIBUTOS
    # .........
    atributos = data[['NOMBRE', 'ALTITUD', 'NOM_PROV', 'LONGITUD', 'LATITUD', 'DATUM']].iloc[0]
    
    # SERIE DIARIA
    # ...........
    # fechas de la serie
    start = pd.datetime(data.AÑO.iloc[0], data.MES.iloc[0], 1).date()
    year, month = data.AÑO.iloc[-1], data.MES.iloc[-1]
    end = pd.datetime(year, month, monthrange(year, month)[1]).date()
    fechas = pd.date_range(start, end)
    # ordenar datos en una serie
    serie = pd.Series(index=fechas)
    cols = ['P' + str(i) for i in np.arange(1, 32)]
    for idx in data.index:
        year, month = data.loc[idx, ['AÑO', 'MES']]
        days = monthrange(year, month)[1]
        d = pd.date_range(pd.datetime(year, month, 1), pd.datetime(year, month, days))
        serie[d] = data.loc[idx, cols[:days]]
    # corregir serie
    serie[serie == -3] = 0
    serie[serie == -4] = np.nan
    serie /= 10
        
    return serie, atributos

### Temperatura

Código para dividir algunos de los archivos de la AEMET en los que vienen varias provincias.

```Python
ruta = 'G:/CLIMA/02_HYDRO-CLIMATE/AEMET/temperatura/'
aux = pd.read_csv(ruta + 'Zamora,Teruel,Toledo,Valencia,Valladolid,Zaragoza.csv', sep=';', encoding='latin1', index_col='INDICATIVO')

provincias = aux.NOM_PROV.unique()

for provincia in provincias:
    aux_ = aux.loc[aux.NOM_PROV == provincia,:]
    palabras = provincia.split()
    filename = ' '.join([palabra[0] + palabra[1:].lower() for palabra in palabras]) + '.csv'
    aux_.to_csv(ruta + filename, sep=';', float_format='%.0f')
```

In [189]:
def AEMET_Tdiaria(estacion, provincia):
    """Extrae las series diarias de temperatura (máxima y mínima) de la AEMET.
    
    Parámetros:
    -----------
    estacion:  string. Código de la estación
    provinvia: string. Provincia a la que pertenece la estación. Define el archivo csv donde acceder a los datos, por lo que debe replicar 
                       el nombre del archivo, es decir, primera letra en mayúsucula y sin ninguna tilde.
                       
    Salidas:
    --------
    Tmax:      pd.Series. Serie diaria de temperatura máxima (ºC)
    Tmin:      pd.Series. Serie diaria de temperatura mínima (ºC)
    atributos: pd.Series. Atributos de la estación: coordenadas, altitud...
    """
    
    # cargar datos
    data = pd.read_csv('G:/CLIMA/02_HYDRO-CLIMATE/AEMET/temperatura/' + provincia + '.csv', sep=';', index_col=0)
    data = data.loc[estacion, :]
    
    # atributos
    atributos = data[['NOMBRE', 'ALTITUD', 'NOM_PROV', 'LONGITUD', 'LATITUD', 'DATUM']].iloc[0]
    
    # series vacias de Tmin y Tmax
    start = pd.datetime(data.AÑO.iloc[0], data.MES.iloc[0], 1).date()
    añof, mesf = data.AÑO.iloc[-1], data.MES.iloc[-1]
    end = pd.datetime(añof, mesf, monthrange(añof, mesf)[1]).date()
    Tmin = pd.Series(index=pd.date_range(start, end))
    Tmax = pd.Series(index=pd.date_range(start, end))
    
    # completar las series
    for i in range(data.shape[0] - 1):
        año, mes = data.AÑO.iloc[i], data.MES.iloc[i]
        dias = monthrange(año, mes)[1]
        fechas = pd.date_range(pd.datetime(año, mes, 1).date(), pd.datetime(año, mes, dias))
        colMax = ['TMAX' + str(n) for n in range(1, dias + 1)]
        colMin = ['TMIN' + str(n) for n in range(1, dias + 1)]
        Tmax[fechas] = data.iloc[i][colMax]
        Tmin[fechas] = data.iloc[i][colMin]
    
    # convertir décimas de grado en grado
    Tmax /= 10
    Tmin /= 10
    
    return Tmax, Tmin, atributos